In [474]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

In [475]:
dataset_path = "../data/raw/dataset.xlsx"

In [476]:
df = pd.read_excel(dataset_path)
df.head()

,Variable,CODE,age.diag,age.menarc,age.menop,education,marital,gravity,lactaton,fpregnancy,...,hrt,smoking,personal.Hx,Family,agef,weight,height,lifeevent,stage,G
0,Label,NaN,age at diagnose,age of menarce,age of menopause,education,marital,gravity,lactation duration,age at first pregnancy,...,duration of HRT use,NaN,personal history of breast CA,NaN,age of involved with breast cancer,weight,height,NaN,NaN,Grade
1,NaN,NaN,41,12,NaN,3,2,1,27,20,...,0,0.0,1,4.0,38,69,158,0.0,3a,2
2,NaN,NaN,57,15,45,2,4,1,48,22,...,0,2.0,0,0.0,0,89,157,0.0,NaN,2
3,NaN,NaN,37,NaN,NaN,2,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,NaN,NaN,72,13,42,4,2,1,6,32,...,0,0.0,1,5.0,32,53,154,0.0,NaN,1


In [477]:
print(len(df))
print(df.columns)

1002
Index(['Variable', 'CODE', 'age.diag', 'age.menarc', 'age.menop', 'education',
       'marital', 'gravity', 'lactaton', 'fpregnancy', 'infertility',
       'reproduct', 'ocpuse', 'ocp', 'hrt', 'smoking', 'personal.Hx', 'Family',
       'agef', 'weight', 'height', 'lifeevent', 'stage', 'G'],
      dtype='object')


In [478]:
df = df.drop(df.index[0])
df.head()

,Variable,CODE,age.diag,age.menarc,age.menop,education,marital,gravity,lactaton,fpregnancy,...,hrt,smoking,personal.Hx,Family,agef,weight,height,lifeevent,stage,G
1,NaN,NaN,41,12,NaN,3,2,1,27,20,...,0,0.0,1,4.0,38,69,158,0.0,3a,2
2,NaN,NaN,57,15,45,2,4,1,48,22,...,0,2.0,0,0.0,0,89,157,0.0,NaN,2
3,NaN,NaN,37,NaN,NaN,2,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,NaN,NaN,72,13,42,4,2,1,6,32,...,0,0.0,1,5.0,32,53,154,0.0,NaN,1
5,NaN,NaN,37,NaN,NaN,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,63,160,NaN,NaN,1


In [479]:
df.dtypes

Variable        object
CODE           float64
age.diag        object
age.menarc      object
age.menop       object
education       object
marital         object
gravity         object
lactaton        object
fpregnancy      object
infertility     object
reproduct       object
ocpuse         float64
ocp             object
hrt             object
smoking        float64
personal.Hx     object
Family         float64
agef            object
weight          object
height          object
lifeevent      float64
stage           object
G               object
dtype: object

In [480]:
cleaned_df = df.copy()

In [481]:
code_dist = range(int(df['CODE'].max()), (int(df['CODE'].max()) + df['CODE'].isna().sum() + 1))
print(cleaned_df['CODE'].isna().sum())
# Replace NaN values from code_dist
cleaned_df['CODE'] = cleaned_df['CODE'].fillna(pd.Series(code_dist))
print(cleaned_df['CODE'].isna().sum())
cleaned_df['CODE'] = cleaned_df['CODE'].astype(int)
print(cleaned_df[cleaned_df.duplicated(['CODE'])]['CODE'])

14
0
70      6001486
171     8000369
664    83001071
Name: CODE, dtype: int64


In [482]:
age_cols = [col for col in df.columns if col.startswith('age')]
print(cleaned_df[age_cols].isna().sum())
for col in age_cols:
    print("---" * 4)
    print(col, df[col].dtype)
    print(cleaned_df[col].unique())

age.diag        4
age.menarc     49
age.menop     562
agef          957
dtype: int64
------------
age.diag object
[41 57 37 72 53 39 50 58 40 34 67 35 46 43 48 30 32 60 38 54 51 44 29 27
 52 55 42 31 63 71 25 49 45 64 33 nan 28 56 62 79 47 61 68 36 66 26 65 59
 24 75 70 73 74 78 76 69 87 77 83 82 81 18]
------------
age.menarc object
[12 15 nan 13 14 10 16 11 18 9 20 17 8]
------------
age.menop object
[nan 45 42 0 50 39 48 43 34 40 52 46 53 56 44 60 47 30 57 49 51 54 41 33
 55 32 36 59 38 35 37 62]
------------
agef object
['38' '0' nan '32' '40' '36' '42' '70' '60' '45' '29' '30' '37' '59' '35-'
 '39' '50' '41' '24' '57' '55' '80' '46' '65']


In [483]:
def fill_na_with_mean(df, col):
    print("---" * 4)
    print(col)
    print(df[col].unique())
    df[col] = df[col].fillna(0)
    df[col] = df[col].astype(str).str.replace('-', '')
    df[col] = df[col].astype(int)
    df[col] = df[col].replace(0, int(df[df[col] != 0][col].mean()))
    print("-")
    print(df[col].unique())
    return df

In [484]:
for col in age_cols:
    cleaned_df = fill_na_with_mean(cleaned_df, col)

------------
age.diag
[41 57 37 72 53 39 50 58 40 34 67 35 46 43 48 30 32 60 38 54 51 44 29 27
 52 55 42 31 63 71 25 49 45 64 33 nan 28 56 62 79 47 61 68 36 66 26 65 59
 24 75 70 73 74 78 76 69 87 77 83 82 81 18]
-
[41 57 37 72 53 39 50 58 40 34 67 35 46 43 48 30 32 60 38 54 51 44 29 27
 52 55 42 31 63 71 25 49 45 64 33 28 56 62 79 47 61 68 36 66 26 65 59 24
 75 70 73 74 78 76 69 87 77 83 82 81 18]
------------
age.menarc
[12 15 nan 13 14 10 16 11 18 9 20 17 8]
-
[12 15 13 14 10 16 11 18  9 20 17  8]
------------
age.menop
[nan 45 42 0 50 39 48 43 34 40 52 46 53 56 44 60 47 30 57 49 51 54 41 33
 55 32 36 59 38 35 37 62]
-
[47 45 42 50 39 48 43 34 40 52 46 53 56 44 60 30 57 49 51 54 41 33 55 32
 36 59 38 35 37 62]
------------
agef
['38' '0' nan '32' '40' '36' '42' '70' '60' '45' '29' '30' '37' '59' '35-'
 '39' '50' '41' '24' '57' '55' '80' '46' '65']
-
[38 44 32 40 36 42 70 60 45 29 30 37 59 35 39 50 41 24 57 55 80 46 65]


In [485]:
for col in ['education', 'marital', 'gravity']:
    print("---" * 4)
    print(col, cleaned_df[col].dtype)
    print(cleaned_df[col].unique())
    print(cleaned_df[col].isna().sum())
    print(cleaned_df[col].value_counts())

------------
education object
[3 2 4 1 nan]
25
2    340
3    300
4    178
1    158
Name: education, dtype: int64
------------
marital object
[2 4 nan 3 1]
19
2    810
3    111
1     59
4      2
Name: marital, dtype: int64
------------
gravity object
[1 nan 0 2]
72
1    872
2     36
0     21
Name: gravity, dtype: int64


In [486]:
for col in ['education', 'marital', 'gravity']:
    cleaned_df[col].fillna(cleaned_df[col].mode()[0], inplace=True)
    cleaned_df[col] = cleaned_df[col].astype('category')

In [487]:
for col in ['education', 'marital', 'gravity']:
    print("---" * 4)
    print(col, cleaned_df[col].dtype)
    print(cleaned_df[col].unique())
    print(cleaned_df[col].isna().sum())
    print(cleaned_df[col].value_counts())

------------
education category
[3, 2, 4, 1]
Categories (4, int64): [1, 2, 3, 4]
0
2    365
3    300
4    178
1    158
Name: education, dtype: int64
------------
marital category
[2, 4, 3, 1]
Categories (4, int64): [1, 2, 3, 4]
0
2    829
3    111
1     59
4      2
Name: marital, dtype: int64
------------
gravity category
[1, 0, 2]
Categories (3, int64): [0, 1, 2]
0
1    944
2     36
0     21
Name: gravity, dtype: int64


In [488]:
print(cleaned_df['lactaton'].unique())
cleaned_df['lactaton'] = cleaned_df['lactaton'].fillna(int(cleaned_df['lactaton'].mean(skipna=True)))
cleaned_df['lactaton'] = cleaned_df['lactaton'].astype(int)
print(cleaned_df['lactaton'].unique())

[27 48 nan 6 0 96 36 72 2 84 52 78 18 168 60 19 216 24 10 15 9 30 180 144
 56 11 3 4 8 25 12 1 42 29 66 192 120 54 5 50 132 92 28 80 16 40 20 174 13
 90 64 65 17 53 89 108 22 14 7 39 100 55 75 102 33 21 44 51 58 133 79 61
 41 114 43 32 240 118 38 162 45 46 171 86 204 88 138 26 126 288 113 62 156
 68 82 35]
[ 27  48  53   6   0  96  36  72   2  84  52  78  18 168  60  19 216  24
  10  15   9  30 180 144  56  11   3   4   8  25  12   1  42  29  66 192
 120  54   5  50 132  92  28  80  16  40  20 174  13  90  64  65  17  89
 108  22  14   7  39 100  55  75 102  33  21  44  51  58 133  79  61  41
 114  43  32 240 118  38 162  45  46 171  86 204  88 138  26 126 288 113
  62 156  68  82  35]


In [489]:
print(cleaned_df['fpregnancy'].unique())
cleaned_df['fpregnancy'] = cleaned_df['fpregnancy'].replace(0, np.nan)
cleaned_df['fpregnancy'] = cleaned_df['fpregnancy'].fillna(int(cleaned_df['fpregnancy'].mean(skipna=True)))
cleaned_df['fpregnancy'] = cleaned_df['fpregnancy'].astype(int)
print(cleaned_df['fpregnancy'].unique())

[20 22 nan 32 21 18 28 24 17 19 16 23 15 40 13 34 25 29 31 26 14 33 30 35
 27 36 45 41 37 42 38 0]
[20 22 21 32 18 28 24 17 19 16 23 15 40 13 34 25 29 31 26 14 33 30 35 27
 36 45 41 37 42 38]


In [490]:
print(cleaned_df['infertility'].unique())
# fill NaN values with the most frequent value
cleaned_df['infertility'] = cleaned_df['infertility'].fillna(cleaned_df['infertility'].mode()[0])
cleaned_df['infertility'] = cleaned_df['infertility'].astype('category')
print(cleaned_df['infertility'].unique())

[0 nan 1]
[0, 1]
Categories (2, int64): [0, 1]


In [491]:
print(cleaned_df['reproduct'].unique())
cleaned_df['reproduct'] = cleaned_df['reproduct'].fillna(cleaned_df['reproduct'].mode()[0])
cleaned_df['reproduct'] = cleaned_df['reproduct'].astype('category')
print(cleaned_df['reproduct'].unique())

[1 3 nan 2 4]
[1, 3, 2, 4]
Categories (4, int64): [1, 2, 3, 4]


In [492]:
print(cleaned_df['ocpuse'].unique())
cleaned_df['ocpuse'] = cleaned_df['ocpuse'].fillna(cleaned_df['ocpuse'].mode()[0])
cleaned_df['ocpuse'] = cleaned_df['ocpuse'].astype(int)
cleaned_df['ocpuse'] = cleaned_df['ocpuse'].astype('category')
print(cleaned_df['ocpuse'].unique())

[ 0. nan  1.]
[0, 1]
Categories (2, int64): [0, 1]


In [493]:
print(cleaned_df['ocp'].unique())
cleaned_df['ocp'] = cleaned_df['ocp'].fillna(int(cleaned_df['ocp'].mean(skipna=True)))
cleaned_df['ocp'] = cleaned_df['ocp'].astype(int)
print(cleaned_df['ocp'].unique())

[nan 0 24 120 6 84 216 12 300 1 3 252 30 72 168 60 4 36 8 66 96 48 132 108
 7 18 144 180 228 15 156 5 240 276 204 82 2 54 42 52 78 80 288 192 14 90
 10 186 41 100 73 45 202]
[ 61   0  24 120   6  84 216  12 300   1   3 252  30  72 168  60   4  36
   8  66  96  48 132 108   7  18 144 180 228  15 156   5 240 276 204  82
   2  54  42  52  78  80 288 192  14  90  10 186  41 100  73  45 202]


In [494]:
print(cleaned_df['hrt'].unique())
cleaned_df['hrt'] = cleaned_df['hrt'].fillna(int(cleaned_df['hrt'].mean(skipna=True)))
cleaned_df['hrt'] = cleaned_df['hrt'].astype(int)
print(cleaned_df['hrt'].unique())

[0 nan 2 6 24 12 96 4 36 3 30 18 72 48 5 60 9 8 54]
[ 0  1  2  6 24 12 96  4 36  3 30 18 72 48  5 60  9  8 54]


In [495]:
print(cleaned_df['smoking'].unique())
cleaned_df['smoking'] = cleaned_df['smoking'].fillna(cleaned_df['smoking'].mode()[0])
cleaned_df['smoking'] = cleaned_df['smoking'].replace(12, cleaned_df['smoking'].mode()[0])
cleaned_df['smoking'] = cleaned_df['smoking'].astype(int)
cleaned_df['smoking'] = cleaned_df['smoking'].astype('category')
print(cleaned_df['smoking'].unique())


[ 0.  2. nan  1. 12.]
[0, 2, 1]
Categories (3, int64): [0, 1, 2]


In [496]:
print(cleaned_df['personal.Hx'].unique())
cleaned_df['personal.Hx'] = cleaned_df['personal.Hx'].fillna(cleaned_df['personal.Hx'].mode()[0])
cleaned_df['personal.Hx'] = cleaned_df['personal.Hx'].astype('category')
print(cleaned_df['personal.Hx'].unique())

[1 0 nan]
[1, 0]
Categories (2, int64): [0, 1]


In [497]:
valid_choices = [i for i in range(0, 8)]
print(cleaned_df['Family'].unique())
print(cleaned_df['Family'].value_counts())
# fill NaN values with the most frequent value
cleaned_df['Family'] = cleaned_df['Family'].fillna(cleaned_df['Family'].mode()[0])
# replace invalid values with the most frequent value
cleaned_df['Family'] = cleaned_df['Family'].map(lambda x: cleaned_df['Family'].mode()[0] if x not in valid_choices else x)
cleaned_df['Family'] = cleaned_df['Family'].astype(int)
cleaned_df['Family'] = cleaned_df['Family'].astype('category')
print(cleaned_df['Family'].unique())

[ 4.  0. nan  5.  6.  2.  1. 23.  3. 13.  7.]
0.0     829
3.0      62
1.0      61
4.0       6
13.0      6
2.0       5
5.0       3
6.0       3
23.0      2
7.0       1
Name: Family, dtype: int64
[4, 0, 5, 6, 2, 1, 3, 7]
Categories (8, int64): [0, 1, 2, 3, 4, 5, 6, 7]


In [498]:
print(cleaned_df['weight'].unique())
cleaned_df['weight'] = cleaned_df['weight'].fillna(int(cleaned_df['weight'].mean(skipna=True)))
cleaned_df['weight'] = cleaned_df['weight'].astype(int)
print(cleaned_df['weight'].unique())

[69 89 nan 53 63 55 85 68 95 65 75 58 70 78 86 64 71 43 50 48 61 67 60 35
 49 59 76 57 56 74 52 45 54 72 51 100 80 90 62 77 79 46 66 73 40 84 81 93
 88 38 83 36 41 105 92 99 44 96 82 104 103 94 101 102 87 114 113 97 98 111
 112 107 42 91 108 115]
[ 69  89  53  63  55  85  68  95  65  75  58  70  78  86  64  71  43  50
  48  61  67  60  35  49  59  76  57  56  74  52  45  54  72  51 100  80
  90  62  77  79  46  66  73  40  84  81  93  88  38  83  36  41 105  92
  99  44  96  82 104 103  94 101 102  87 114 113  97  98 111 112 107  42
  91 108 115]


In [499]:
print(cleaned_df['height'].unique())
cleaned_df['height'] = cleaned_df['height'].fillna(int(cleaned_df['height'].mean(skipna=True)))
cleaned_df['height'] = cleaned_df['height'].astype(int)


[158 157 nan 154 160 165 169 164 152 145 155 151 153 147 150 166 156 167
 132 168 161 159 163 144 148 162 149 146 130 143 171 178 124 142 173 172
 174 140 170 120 175 141 138 137]


In [500]:
valid_choices = [i for i in range(0, 11)]
print(cleaned_df['lifeevent'].unique())
print(cleaned_df['lifeevent'].value_counts())
print(cleaned_df['lifeevent'].isna().sum())
# fill NaN values with the most frequent value
cleaned_df['lifeevent'] = cleaned_df['lifeevent'].fillna(cleaned_df['lifeevent'].mode()[0])
# replace invalid values with the most frequent value
cleaned_df['lifeevent'] = cleaned_df['lifeevent'].map(lambda x: cleaned_df['lifeevent'].mode()[0] if x not in valid_choices else x)
cleaned_df['lifeevent'] = cleaned_df['lifeevent'].astype(int)
cleaned_df['lifeevent'] = cleaned_df['lifeevent'].astype('category')
print(cleaned_df['lifeevent'].unique())

[ 0. nan  6.  1. 36.]
0.0     16
6.0      1
1.0      1
36.0     1
Name: lifeevent, dtype: int64
982
[0, 6, 1]
Categories (3, int64): [0, 1, 6]


In [501]:
print(cleaned_df['stage'].unique())
# count NaN values
print(cleaned_df['stage'].isna().sum())
# drop rows with NaN stage values
cleaned_df = cleaned_df.dropna(subset=['stage'])

['3a' nan '2b' '1' '3c' '2a' '4' '0' '3b']
385


In [502]:
print(cleaned_df['G'].unique())
# count NaN values
print(cleaned_df['G'].isna().sum())
# drop rows with NaN G values
cleaned_df = cleaned_df.dropna(subset=['G'])

[2 nan 1 3]
171


In [503]:
# count total values
print(len(cleaned_df))

445


In [504]:
# count NaN values
print(cleaned_df.isna().sum())

Variable       445
CODE             0
age.diag         0
age.menarc       0
age.menop        0
education        0
marital          0
gravity          0
lactaton         0
fpregnancy       0
infertility      0
reproduct        0
ocpuse           0
ocp              0
hrt              0
smoking          0
personal.Hx      0
Family           0
agef             0
weight           0
height           0
lifeevent        0
stage            0
G                0
dtype: int64


In [505]:
cleaned_df.to_csv('../data/processed/cleaned_data.csv', index=True)